In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import gensim

In [2]:
#pip install -U gensim

In [43]:
import nltk
from nltk.corpus import stopwords
import re
import joblib
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [4]:
FoodTweets = pd.read_csv("C:/Users/User/Documents/GitHub/MSC_DA_CA2v4/ProcessFilteredTweets.csv", error_bad_lines=False)

b'Skipping line 1904: expected 6 fields, saw 8\nSkipping line 2003: expected 6 fields, saw 7\nSkipping line 4594: expected 6 fields, saw 7\n'


In [5]:
FoodTweets.head(5)

,column1,column2,column3,column4,column5,column6
0,0,1.678358e+09,Sat May 02 06:26:48 PDT 2009,NO_QUERY,kconnxx,glad to know my family bought tons of gluten f...
1,0,1.827494e+09,Sun May 17 10:49:49 PDT 2009,NO_QUERY,MarieLuv,@pornlaw damn u on the grill when i'm in vegas...
2,0,1.834532e+09,Mon May 18 03:17:33 PDT 2009,NO_QUERY,angelynewolfe,tofu too healthy... searching through the cupb...
3,0,1.963275e+09,Fri May 29 12:28:39 PDT 2009,NO_QUERY,UlyssesAR,looking for food at home!.... nothing
4,0,1.976219e+09,Sat May 30 16:50:48 PDT 2009,NO_QUERY,Kelsayy237,watching something on food network with corey....


In [6]:
FoodTweets.rename(columns = {'column1':'Sentiment', 'column2':'ids',
                              'column3':'date','column4':'flag','column5':'user','column6':'tweet'}, inplace = True)                                           

In [7]:
FoodTweets.head(5)

,Sentiment,ids,date,flag,user,tweet
0,0,1.678358e+09,Sat May 02 06:26:48 PDT 2009,NO_QUERY,kconnxx,glad to know my family bought tons of gluten f...
1,0,1.827494e+09,Sun May 17 10:49:49 PDT 2009,NO_QUERY,MarieLuv,@pornlaw damn u on the grill when i'm in vegas...
2,0,1.834532e+09,Mon May 18 03:17:33 PDT 2009,NO_QUERY,angelynewolfe,tofu too healthy... searching through the cupb...
3,0,1.963275e+09,Fri May 29 12:28:39 PDT 2009,NO_QUERY,UlyssesAR,looking for food at home!.... nothing
4,0,1.976219e+09,Sat May 30 16:50:48 PDT 2009,NO_QUERY,Kelsayy237,watching something on food network with corey....


In [8]:
FoodTweets['Polarity'] = FoodTweets.loc[:, 'Sentiment']

In [9]:
### Removing the unnessary columns
FoodTweetsa = FoodTweets [['Sentiment','tweet','date','Polarity']]
FoodTweetsa.head(5)

,Sentiment,tweet,date,Polarity
0,0,glad to know my family bought tons of gluten f...,Sat May 02 06:26:48 PDT 2009,0
1,0,@pornlaw damn u on the grill when i'm in vegas...,Sun May 17 10:49:49 PDT 2009,0
2,0,tofu too healthy... searching through the cupb...,Mon May 18 03:17:33 PDT 2009,0
3,0,looking for food at home!.... nothing,Fri May 29 12:28:39 PDT 2009,0
4,0,watching something on food network with corey....,Sat May 30 16:50:48 PDT 2009,0


In [10]:
FoodTweetsa.Polarity.replace({0:'Negative',2:'Neutral',4:'Positive'},inplace=True)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [11]:
FoodTweetsa.head(5)

,Sentiment,tweet,date,Polarity
0,0,glad to know my family bought tons of gluten f...,Sat May 02 06:26:48 PDT 2009,Negative
1,0,@pornlaw damn u on the grill when i'm in vegas...,Sun May 17 10:49:49 PDT 2009,Negative
2,0,tofu too healthy... searching through the cupb...,Mon May 18 03:17:33 PDT 2009,Negative
3,0,looking for food at home!.... nothing,Fri May 29 12:28:39 PDT 2009,Negative
4,0,watching something on food network with corey....,Sat May 30 16:50:48 PDT 2009,Negative


In [12]:
FoodTweetsa.to_csv('FoodTweets.csv')

In [13]:
stop_words=set(stopwords.words('english'))
stop_words.remove('not')

In [14]:
corpus=[]
for i in range(0,len(FoodTweets)):
    review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',FoodTweetsa['tweet'][i])
    review=review.lower()
    review=review.split()
    review=[word for word in review if not word in stop_words]
    review=' '.join(review)
    corpus.append(review)

In [15]:
FoodTweetsa.tweet = corpus

C:\Users\User\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(FoodTweetsa,test_size=0.20,random_state=123)

In [17]:
train_df.head()

,Sentiment,tweet,date,Polarity
1369,0,weehee wait get home though food u lookin gooo...,Wed Jun 03 09:08:47 PDT 2009,Negative
1172,0,well not right low food budget money,Sat Jun 06 19:15:19 PDT 2009,Negative
3848,0,keep eating junk food,Mon Jun 15 21:08:38 PDT 2009,Negative
2686,0,never eating applebees food unsatisfying stoma...,Wed Jun 17 21:12:03 PDT 2009,Negative
7006,4,chillin family getting tacos ochoa best food w...,Sat May 09 19:05:41 PDT 2009,Positive


In [18]:
test_df.head()

,Sentiment,tweet,date,Polarity
697,0,know gotta get food get work,Sun May 03 17:19:10 PDT 2009,Negative
6875,4,currently looking around food got drunken munc...,Sun Jun 07 16:18:54 PDT 2009,Positive
3508,0,problem away home 4 weeks amp returning sub ze...,Fri Jun 19 17:37:53 PDT 2009,Negative
7386,4,thanks putting us today foodbuzz top 9 guess l...,Fri Jun 05 08:56:19 PDT 2009,Positive
798,0,thinking lunch soon food,Fri May 29 08:07:14 PDT 2009,Negative


In [19]:
documents = [text.split() for text in train_df.tweet]

In [20]:
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=300, 
                                            window=7, 
                                            min_count=10, 
                                            workers=8)

In [21]:
w2v_model.build_vocab(documents)

In [22]:
vocab_len = len(w2v_model.wv)
print(vocab_len)

922


In [23]:
#words = w2v_model.wv.vocab.keys()
#vocab_size = len(words)
#print("Vocab size", vocab_size)

In [24]:
w2v_model.train(documents, total_examples=len(documents), epochs=30)

(1026936, 1926090)

In [25]:
w2v_model.wv.most_similar("good")

[('fun', 0.8514115214347839),
 ('times', 0.8468332886695862),
 ('cute', 0.8109042644500732),
 ('friends', 0.8045111298561096),
 ('music', 0.7898163199424744),
 ('great', 0.789203405380249),
 ('beer', 0.7827014923095703),
 ('beautiful', 0.7568893432617188),
 ('playing', 0.7551497220993042),
 ('sunshine', 0.7547372579574585)]

In [26]:
w2v_model.wv.most_similar("hate")

[('damn', 0.9096203446388245),
 ('wanted', 0.8615754246711731),
 ('ugh', 0.8560574650764465),
 ('cat', 0.8474496006965637),
 ('lose', 0.8291378021240234),
 ('something', 0.8283616304397583),
 ('want', 0.8244571685791016),
 ('dont', 0.8241872787475586),
 ('havent', 0.8177163004875183),
 ('cant', 0.813858687877655)]

In [27]:
w2v_model.wv.most_similar("great")

[('awesome', 0.9227442741394043),
 ('music', 0.9087857604026794),
 ('weather', 0.8987402319908142),
 ('lovely', 0.8933026790618896),
 ('company', 0.8906828761100769),
 ('beer', 0.888113796710968),
 ('friends', 0.8567461967468262),
 ('beautiful', 0.8483742475509644),
 ('evening', 0.8453469276428223),
 ('festival', 0.8386942148208618)]

In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df.tweet)

In [29]:
tokenizer.word_index

{'food': 1,
 'good': 2,
 'not': 3,
 'get': 4,
 'amp': 5,
 'eat': 6,
 'go': 7,
 'like': 8,
 'hungry': 9,
 'day': 10,
 'need': 11,
 'going': 12,
 'lol': 13,
 'quot': 14,
 'today': 15,
 'home': 16,
 'love': 17,
 'foods': 18,
 'eating': 19,
 'got': 20,
 'time': 21,
 'great': 22,
 'much': 23,
 'im': 24,
 'want': 25,
 'back': 26,
 'really': 27,
 'work': 28,
 'think': 29,
 'night': 30,
 'chinese': 31,
 'dinner': 32,
 'well': 33,
 'ate': 34,
 'make': 35,
 '2': 36,
 'u': 37,
 'still': 38,
 'lunch': 39,
 'house': 40,
 'feel': 41,
 'oh': 42,
 'one': 43,
 'haha': 44,
 'know': 45,
 'yummy': 46,
 'last': 47,
 'bad': 48,
 'poisoning': 49,
 'miss': 50,
 'better': 51,
 'right': 52,
 'nice': 53,
 'tonight': 54,
 'fun': 55,
 'gonna': 56,
 'though': 57,
 'shopping': 58,
 'way': 59,
 'junk': 60,
 'free': 61,
 'went': 62,
 'see': 63,
 'tomorrow': 64,
 'fast': 65,
 'bring': 66,
 'getting': 67,
 'seafood': 68,
 'would': 69,
 'people': 70,
 'best': 71,
 'awesome': 72,
 'hope': 73,
 'w': 74,
 'sick': 75,
 'thai

In [30]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

11132

In [31]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df.tweet), maxlen=300)
X_train

array([[    0,     0,     0, ...,  4361,   207,   650],
       [    0,     0,     0, ...,     1,  1291,   160],
       [    0,     0,     0, ...,    19,    60,     1],
       ...,
       [    0,     0,     0, ...,   599,    37,  2634],
       [    0,     0,     0, ..., 11131,     1,   223],
       [    0,     0,     0, ...,    88,  3998,   237]])

In [32]:
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df.tweet), maxlen=300)
X_test

array([[    0,     0,     0, ...,     1,     4,    28],
       [    0,     0,     0, ...,    37,    25,   660],
       [    0,     0,     0, ...,     1,    48,   635],
       ...,
       [    0,     0,     0, ...,   139, 10516,  2005],
       [    0,     0,     0, ...,  5907,  1521,  1030],
       [    0,     0,     0, ...,    86,   117,     1]])

In [33]:
y_train=train_df.Sentiment
y_train.head()

1369    0
1172    0
3848    0
2686    0
7006    4
Name: Sentiment, dtype: int64

In [34]:
y_test=test_df.Sentiment
y_test.head()

697     0
6875    4
3508    0
7386    4
798     0
Name: Sentiment, dtype: int64

In [35]:
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_test=labelencoder.fit_transform(y_test)

In [36]:
y_train.shape

(6742,)

In [37]:
y_test.shape

(1686,)

In [38]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(11132, 300)


In [39]:
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)

In [40]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(1, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          3339600   
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 1)                 1208      
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 3,340,810
Trainable params: 1,210
Non-trainable params: 3,339,600
_________________________________________________________________


In [41]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [44]:
early_stopping= EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True)

In [45]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7, batch_size=32,callbacks=[early_stopping])

Epoch 1/7
211/211 [==============================] - 285s 1s/step - loss: 0.6588 - accuracy: 0.6055 - val_loss: 0.6143 - val_accuracy: 0.6690
Epoch 2/7
211/211 [==============================] - 250s 1s/step - loss: 0.6211 - accuracy: 0.6531 - val_loss: 0.6021 - val_accuracy: 0.6779
Epoch 3/7
211/211 [==============================] - 239s 1s/step - loss: 0.6140 - accuracy: 0.6639 - val_loss: 0.5967 - val_accuracy: 0.6880
Epoch 4/7
211/211 [==============================] - 230s 1s/step - loss: 0.6112 - accuracy: 0.6707 - val_loss: 0.5954 - val_accuracy: 0.6874
Epoch 5/7
211/211 [==============================] - 144s 678ms/step - loss: 0.6083 - accuracy: 0.6672 - val_loss: 0.5895 - val_accuracy: 0.6957
Epoch 6/7
211/211 [==============================] - 100s 475ms/step - loss: 0.6075 - accuracy: 0.6691 - val_loss: 0.5882 - val_accuracy: 0.6969
Epoch 7/7
211/211 [==============================] - 88s 416ms/step - loss: 0.6046 - accuracy: 0.6704 - val_loss: 0.5857 - val_accuracy: 0.699

In [46]:
_,accuracy_1w=model.evaluate(X_test,y_test)

53/53 [==============================] - 4s 82ms/step - loss: 0.5857 - accuracy: 0.6999


In [48]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30, batch_size=32, callbacks=[early_stopping])

Epoch 1/30
211/211 [==============================] - 108s 514ms/step - loss: 0.6021 - accuracy: 0.6755 - val_loss: 0.5854 - val_accuracy: 0.6981
Epoch 2/30
211/211 [==============================] - 101s 478ms/step - loss: 0.6019 - accuracy: 0.6744 - val_loss: 0.5838 - val_accuracy: 0.7017
Epoch 3/30
211/211 [==============================] - 107s 507ms/step - loss: 0.6013 - accuracy: 0.6749 - val_loss: 0.5815 - val_accuracy: 0.7034
Epoch 4/30
211/211 [==============================] - 102s 483ms/step - loss: 0.6002 - accuracy: 0.6746 - val_loss: 0.5827 - val_accuracy: 0.6886
Epoch 5/30
211/211 [==============================] - 101s 478ms/step - loss: 0.5971 - accuracy: 0.6827 - val_loss: 0.5861 - val_accuracy: 0.6963
Epoch 6/30
211/211 [==============================] - 99s 469ms/step - loss: 0.5979 - accuracy: 0.6821 - val_loss: 0.5815 - val_accuracy: 0.6987
Epoch 7/30
211/211 [==============================] - 101s 480ms/step - loss: 0.5992 - accuracy: 0.6842 - val_loss: 0.5799 - 

In [47]:
_,accuracy_1m=model.evaluate(X_test,y_test)

53/53 [==============================] - 4s 66ms/step - loss: 0.5857 - accuracy: 0.6999


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7, batch_size=32,callbacks=[early_stopping])

In [ ]:
_,accuracy_3m=model.evaluate(X_test,y_test)

In [ ]:
#model_history=model.fit(X_train, y_train,batch_size=1024,epochs=15,validation_split=0.1,verbose=1)

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs=range(len(acc))

In [ ]:
plt.plot(epochs,acc,label='Trainin_acc',color='blue')
plt.plot(epochs,val_acc,label='Validation_acc',color='red')
plt.legend()
plt.title("Training and Validation Accuracy")

In [ ]:
plt.plot(epochs,loss,label='Training_loss',color='blue')
plt.plot(epochs,val_loss,label='Validation_loss',color='red')
plt.legend()
plt.title("Training and Validation loss")

In [ ]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt